In [68]:
import pandas as pd
import numpy as np
import pickle
import tensorflow

In [69]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [70]:
data = data.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [71]:
# LOADING THE ENCODERS

with open('label_encoder_gender.pkl','rb') as file : 
    gender_encoder = pickle.load(file)
with open('one_hot_encoder_geo.pkl','rb') as file:
    geo_encoder = pickle.load(file)
    

In [72]:
data['Gender'] = gender_encoder.transform(data['Gender'])

In [73]:
encoded_geo = geo_encoder.transform(data[['Geography']])


In [74]:
geo_df = pd.DataFrame(encoded_geo.toarray(),columns=geo_encoder.get_feature_names_out())

In [75]:
data = pd.concat([data,geo_df],axis=1)
data = data.drop(columns=['Geography'],axis=1)

In [76]:
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [77]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [78]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_values = scaler.fit_transform(X_test)
scaled_values

array([[-5.12501721e-01,  9.09111664e-01, -6.77299309e-01, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01],
       [-2.36046598e-01,  9.09111664e-01,  3.84298354e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-4.61306328e-01, -1.09997489e+00,  4.80807232e-01, ...,
        -1.02020406e+00, -5.75810666e-01,  1.77464858e+00],
       ...,
       [ 8.59534812e-01, -1.09997489e+00,  7.70333868e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [ 4.70449825e-01,  9.09111664e-01, -9.66825944e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-1.84851205e-01,  9.09111664e-01, -1.73715981e-03, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01]])

In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard ##IMPORTANT
import datetime

In [83]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #linear activation function for default
])


model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
# SET UP THE TENSORBOARD TO PRINT LOGS

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience = 10, restore_best_weights=True)

history = model.fit(
    X_train,y_train, validation_data = (X_test,y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]    
)

Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 70466.6719 - mae: 70466.6719 - val_loss: 65833.0781 - val_mae: 65833.0781
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 63192.7773 - mae: 63192.7773 - val_loss: 58280.7109 - val_mae: 58280.7109
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 54976.6172 - mae: 54976.6172 - val_loss: 51727.9492 - val_mae: 51727.9492
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 52118.6055 - mae: 52118.6055 - val_loss: 51289.9648 - val_mae: 51289.9648
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 51882.8867 - mae: 51882.8867 - val_loss: 50796.0469 - val_mae: 50796.0469
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 51528.7539 - mae: 51528.7539 - val_loss: 50546.6914 - val_mae: 50546.6914
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 51506.5625 - mae: 51506.5625 

In [ ]:
%load_ext tensorboard

In [87]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 11332), started 0:00:18 ago. (Use '!kill 11332' to kill it.)

In [88]:
test_loss , test_mae = model.evaluate(X_test,y_test)
print(f'The Test MAE is : {test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50307.4492 - mae: 50307.4492
The Test MAE is : 50307.44921875


In [89]:
model.save('regressionProblem.h5')

c:\Users\ASUS\Desktop\University of Luxembourg\Udemy\Learning GenAI\NLP\Churn Classification\annclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
